<a href="https://colab.research.google.com/github/azmihasan/webscraping/blob/main/webscraping_handelsblatt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#First, Installing additional required framework 

* Beautiful Soup : https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* Request


In [ ]:
!pip install beautifulsoup4
!pip install requests

#Lets write the code for web scrapping

framework we used to scrap website of https://www.handelsblatt.com/ :


* Beautiful Soup
* Request
* Datetime
* Pandas : https://pandas.pydata.org/docs/index.html



In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd

# define the url from website will be scrapped
url = 'https://www.handelsblatt.com'
# making request to the site with request function
page = requests.get(url).content
# using beautifulsoup to extract the html structure 
soup = BeautifulSoup(page,'html5lib')
# to get preview of scraped html page
html_preview = soup.prettify()
print(html_preview)
fail = []
def webScrapler(soup):
    data = []
    try:
       
      content = soup.body.find('div', attrs={'class':'vhb-content'})
      '''
      finding all articles with 'a' tags in html side
      '''
      
      articles = content.find_all('a')
      for article in articles:
          
          try:
           
            '''
            extracting all links from attribute of each article 
            '''
            link = article.get('href')
            link = link.strip()
            title = None
            text = None

            
            try:
              if link == None :
              
                fail.append((str(article), 'link failed', datetime.now()))
                
              else :
    
                try:
                    
                    if article.find('h1') != None :
                      
                        title = article.find('h1')
                        title = str(title.text).strip()
                      
                    elif article.find('h3') != None :
                      
                        title = article.find('h3')
                        title = str(title.text).strip()
                        title = title.split()
                        title = ' '.join(title)
                        
                    
                    elif article.find('span', class_ = 'vhb-headline') != None:
                        title = article.find('span', class_ = 'vhb-headline')
                        title = str(title.text).strip()
                except:
                    fail.append((str(article), 'title failed', datetime.now()))
                    
                try :  
                    
                    text = article.find('p')
                    
                    if text == None :
                        
                        fail.append((str(article), 'text failed', datetime.now()))
                      
                    else:
                        
                        string = str(text.text)
                        string = string.strip()
                        string = string.split()
                        string = ' '.join(string)
                        text = string
                        
                except:
                    fail.append((str(article), 'text failed', datetime.now()))
                    
            except:              
                fail.append((str(article), 'link failed', datetime.now())) 
            
            if 'http' not in link:
              link = url + str(link)
              data.append((title, text, link, datetime.now() ,str(article)))
            elif '#teilen' in link or '#' in link:
              link = url
              data.append((title, text, link, datetime.now() ,str(article)))
            else:
              data.append((title, text, link, datetime.now() ,str(article)))  
    
          except:
            fail.append((str(article), 'link failed', datetime.now()))
            
      
    except:
      print('No Finding Any')
      
    print('webscraping procced')
    return data

data = webScrapler(soup)

<!DOCTYPE html>
<html class="no-js" data-cm-metadata="" lang="de">
 <head>
  <link href="https://app.handelsblatt.com/" media="mobile" rel="alternate"/>
  <script type="text/javascript">
  </script>
  <meta content="No author" name="parsely-author"/>
  <script>
   var mobileSwitchHandler = {
    url: null,
    ready: !1,
    init: function(a) {
        "use strict";
        var qs;
        if (null !== (qs = document.querySelector('link[media="mobile"][rel="alternate"][href]')) && (this.url = qs.getAttribute("href")), void 0 === this.url || null === this.url) return !1;
        if ("" !== function(name) {
                name = name.replace(/[\[]/, "\\[").replace(/[\]]/, "\\]");
                var regex = new RegExp("[\\?&]" + name + "=([^&#]*)"),
                    results = regex.exec(location.search);
                return null === results ? "" : decodeURIComponent(results[1].replace(/\+/g, " "))
            }("home_teaser") || !this.isUrlWhitelisted(window.location.href)) return

Identify failed scraping

Not all attributes from the failed scraping won't be dismissed but some left scraped attributes still be saved in the data.

In [ ]:
for i in fail:
  print(i[0]+i[1])

len(fail)

<a class="vhb-share-link ajaxify highlight" data-toggle="dropdown" data-width="275" href="#teilen">Teilen</a>text failed
<a class="vhb-share-socialmedia-flipboard ajaxify" data-command="track" data-count="/ajaxentry/nocache/socialinc?elementId=28124144&amp;action=inc&amp;service=flipboard" data-methods="POST" data-share-title="Ukraine: UN-Vollversammlung verurteilt mit überwältigender Mehrheit russische Invasion" data-share-url="https://www.handelsblatt.com/28124144.html" data-track="asd_flipboard" title="Auf Flipboard teilen"></a>link failed
<a class="vhb-share-socialmedia-getpocket ajaxify" data-command="track" data-count="/ajaxentry/nocache/socialinc?elementId=28124144&amp;action=inc&amp;service=getpocket" data-methods="POST" data-share-title="Ukraine: UN-Vollversammlung verurteilt mit überwältigender Mehrheit russische Invasion" data-share-url="https://www.handelsblatt.com/28124144.html" data-track="asd_getpocket" title="In GetPocket speichern"></a>link failed
<a class="vhb-share-s

310

The Result of extracted data before Cleaning and Categorizing

In [ ]:
df =  pd.DataFrame(data, columns = ['Title','Text','Link','Scraping Time','HTML Tag'])
df = df.drop(['HTML Tag'], axis=1)
df.head(10)
result_df = df.drop_duplicates(subset=['Link'], keep='first')
print(len(df))
print(len(result_df))
result_df.head(len(result_df))

237
174


,Title,Text,Link,Scraping Time
0,Ukraine: UN-Vollversammlung verurteilt mit übe...,"141 Staaten nehmen eine Resolution an, die Rus...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.240782
1,None,None,https://www.handelsblatt.com#teilen,2022-03-02 20:27:54.241232
2,None,None,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.241929
3,Lage am Abend Russland erwartet neue Verhandlu...,"In den Städten Charkiw, Mariupol und Cherson w...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.242469
4,+++ Newsblog zum Ukraine-Krieg +++ Rettung von...,Russische Regierung plant Verkaufsverbot von B...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.243339
...,...,...,...,...
232,None,,https://financebriefing.handelsblatt.com/de/,2022-03-02 20:27:54.382434
233,None,,https://assign.handelsblatt.com?service=https%...,2022-03-02 20:27:54.382707
234,None,,https://assign.handelsblatt.com/sso/register?s...,2022-03-02 20:27:54.383033
235,None,,https://www.handelsblatt.com/audio/global-chan...,2022-03-02 20:27:54.383318


#Cleaning  and Categorizing  the data

For data with attribute of Title, Text, Link and Time.

Data will be moved to list of artikel_data and restore in artikel as DataFrame:

In [ ]:
artikel_data = []
for i in data:
  if i[0] != None and i[1] !=None:
    
    artikel_data.append((i[0],i[1],i[2],i[3]))

artikel = pd.DataFrame(artikel_data, columns = ['Title','Text','Link','Scraping Time'])
artikel = artikel.drop_duplicates(subset=['Title','Text', 'Link'], keep='first')
artikel.head(40)
#print(len(artikel))

,Title,Text,Link,Scraping Time
0,Ukraine: UN-Vollversammlung verurteilt mit übe...,"141 Staaten nehmen eine Resolution an, die Rus...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.240782
1,Lage am Abend Russland erwartet neue Verhandlu...,"In den Städten Charkiw, Mariupol und Cherson w...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.242469
2,+++ Newsblog zum Ukraine-Krieg +++ Rettung von...,Russische Regierung plant Verkaufsverbot von B...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.243339
3,"Versorgungskrise Habeck lässt für 1,5 Milliard...",Das Bundeswirtschaftsministerium stockt die Ga...,https://www.handelsblatt.com/politik/deutschla...,2022-03-02 20:27:54.244239
4,Sanktionen Diese russischen Milliardäre stehen...,"Die USA, EU und Großbritannien haben Präsident...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.245866
5,Stärkung der Bundeswehr Rüstungsindustrie bere...,Bundeskanzler Scholz hat massive Rüstungsinves...,https://www.handelsblatt.com/unternehmen/indus...,2022-03-02 20:27:54.246718
6,"Swift-Ausschluss, Vermögensbloackade EU versch...",Mit den Strafen gegen Russlands Notenbank und ...,https://www.handelsblatt.com/finanzen/banken-v...,2022-03-02 20:27:54.247556
7,Ölpreise: Russisches Öl findet keinen Abnehmer...,Der Preisabschlag für das russische „schwarze ...,https://www.handelsblatt.com/finanzen/maerkte/...,2022-03-02 20:27:54.248863
8,Rohstoffe: Ölpreis steigt deutlich über 110 Do...,Der Krieg Russlands in der Ukraine treibt die ...,https://www.handelsblatt.com/finanzen/maerkte/...,2022-03-02 20:27:54.250241
9,Folgen des Ukraine-Kriegs: Die große Trennung?...,Führende Wirtschaftsvertreter bringen erstmals...,https://www.handelsblatt.com/politik/konjunktu...,2022-03-02 20:27:54.251613


For data with attribute of Title, Link and Time without Text

Data will be moved to list of thema_data and restore in thema as DataFrame:

In [ ]:
thema_data = []
for i in data:
  if i[0] != None and i[1] ==None and i[2] != '#teilen' and i[2] != '#':
    thema_data.append((i[0],i[2],i[3]))
thema = pd.DataFrame(thema_data, columns = ['Title','Link','Scraping Time'])
thema = thema.drop_duplicates(subset=['Title', 'Link'], keep='first')
print(len(thema))
thema.head(10)

75


,Title,Link,Scraping Time
0,Leserfavoriten,https://www.handelsblatt.com/das-beste-vom-han...,2022-03-02 20:27:54.244828
1,Ukraine-Krieg sorgt für Engpässe bei Kabelbäum...,https://www.handelsblatt.com/unternehmen/indus...,2022-03-02 20:27:54.287008
2,Auch Mercedes-Benz stoppt Verkauf und Produkti...,https://www.handelsblatt.com/unternehmen/indus...,2022-03-02 20:27:54.287326
3,Russland erwartet neue Verhandlungen am Donner...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.287672
4,Verlängerung des Ausnahmezustands: EU prüft we...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.287976
5,Russisches Öl findet keinen Abnehmer – Westlic...,https://www.handelsblatt.com/finanzen/maerkte/...,2022-03-02 20:27:54.288254
6,"China versucht, Staatsbürger aus Ukraine zu ev...",https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.288582
7,Finanzinvestor KKR trennt sich von Hensoldt-Ak...,https://www.handelsblatt.com/finanzen/anlagest...,2022-03-02 20:27:54.288881
8,UN-Vollversammlung verurteilt mit überwältigen...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.289208
9,Die Ukraine will ihre Wirtschaft im Krieg stab...,https://www.handelsblatt.com/politik/internati...,2022-03-02 20:27:54.289542


For data with attribute of Text, Link and Time without Title

Data will be moved to list of thema_data and restore in thema as DataFrame:

In [ ]:
text_data = []
for i in data:
  if i[0] == None and i[1] != None and i[1] != '':
    text_data.append((i[1],i[2],i[3]))
text = pd.DataFrame(text_data, columns = ['Text','Link','Scraping Time'])
text.head(10)


,Text,Link,Scraping Time
0,Die wichtigsten Neuigkeiten am frühen Morgen m...,https://www.handelsblatt.com/audio/podcast-mor...,2022-03-02 20:27:54.310244
1,In Handelsblatt Today analysieren wir börsentä...,https://www.handelsblatt.com/audio/today/,2022-03-02 20:27:54.310921
2,In Handelsblatt Disrupt diskutiert Sebastian M...,https://www.handelsblatt.com/audio/disrupt-pod...,2022-03-02 20:27:54.311588
3,Professor Michael Hüther analysiert zusammen m...,https://www.handelsblatt.com/audio/economic-ch...,2022-03-02 20:27:54.312229
4,Wie wollen wir in Zukunft arbeiten? Wie führen...,https://www.handelsblatt.com/audio/rethink-wor...,2022-03-02 20:27:54.312902
5,Handelsblatt Green ist der neue Podcast mit Fa...,https://www.handelsblatt.com/audio/green-podcast/,2022-03-02 20:27:54.313605


For data with attribute of Link and Time without Title and Text

Data will be moved to list of thema_data and restore in thema as DataFrame:

In [ ]:
rest_data = []
for i in data:
  if i[0] == None and i[1] == None:
    if i[2] != 'https://www.handelsblatt.com#teilen' and i[2] != 'https://www.handelsblatt.com#':
      rest_data.append((i[2],i[3]))
rest = pd.DataFrame(rest_data, columns = ['Link','Scraping Time'])
rest = rest.drop_duplicates(subset=['Link'], keep='first')

rest.head()

print(len(artikel))
print(len(thema))
print(len(text))
print(len(rest))
total = len(artikel) +  len(thema) + len(text) + len(rest)
print(total)

64
75
6
51
196


#Saving the data as csv in the directory

Export the data to CSV file with pathlib

In [ ]:
result_df_adresses = []
date = datetime.now().strftime('%a %d-%b-%Y : %H %M')
result_df_name = 'rough_data : ' + str(date)
artikel_name = 'artikel_data : ' + str(date)
thema_name = 'thema_data : ' + str(date)
text_name = 'text_data : ' + str(date)
rest_name = 'rest_data : ' + str(date)

result_df_adresses.append(result_df_name)
result_df_adresses.append(artikel_name)
result_df_adresses.append(thema_name)
result_df_adresses.append(text_name)
result_df_adresses.append(rest_name)

result_df.to_csv(result_df_name)
artikel.to_csv(artikel_name)
thema.to_csv(thema_name)
text.to_csv(text_name)
rest.to_csv(rest_name)

adresses = pd.DataFrame(result_df_adresses, columns=['Name'])
adresses.head()

from pathlib import Path

if Path('saved_file').exists():
  adresses.to_csv('saved_file', mode = 'a', header= False)
  print('Yes')
else:
  adresses.to_csv('saved_file')
  print('No')


No


#Opening the saved data

In [ ]:
loaded_file =  pd.read_csv('saved_file')
loaded_file.head(len(loaded_file))

,Unnamed: 0,Name
0,0,rough_data : Wed 02-Mar-2022 : 20 28
1,1,artikel_data : Wed 02-Mar-2022 : 20 28
2,2,thema_data : Wed 02-Mar-2022 : 20 28
3,3,text_data : Wed 02-Mar-2022 : 20 28
4,4,rest_data : Wed 02-Mar-2022 : 20 28


Loading the data from saved file to read the saved CSV-File:

In [ ]:

name = loaded_file['Name']
for i in name: 
  loaded_data = pd.read_csv(i)
  print(len(loaded_data))

174
64
75
6
51
